In [1]:
from scipy.linalg import qr, pinv, solve, norm
from numpy.random import randn
from numpy.linalg import lstsq
import numpy as np

In [2]:
# generate random data matrix
n,d = 6,4
X = randn(n,d)

# optional: give it linearly dependent columns
# X[:,3] = X[:,2]

# Understanding the pseudoinverse

In [3]:
# form pseudoinverse
Xd = pinv(X)

In [4]:
# X†X ≈ I_d
Xd @ X

array([[ 1.00000000e+00, -5.81129381e-17,  4.00349048e-17,
        -5.56362591e-16],
       [-3.42436270e-16,  1.00000000e+00,  4.93354324e-16,
        -8.44210244e-16],
       [-1.48359108e-16,  2.75661069e-16,  1.00000000e+00,
         2.90008685e-16],
       [ 1.12075955e-16,  2.31473581e-16, -6.17905588e-16,
         1.00000000e+00]])

In [5]:
np.allclose(Xd @ X, np.identity(4))

True

In [6]:
# XX† !≈ I_n
X @ Xd

array([[ 0.49565835,  0.15065132,  0.16062101, -0.33267155,  0.30085264,
         0.01742278],
       [ 0.15065132,  0.36378775,  0.16315801, -0.25578872, -0.2890202 ,
        -0.18212448],
       [ 0.16062101,  0.16315801,  0.87344349,  0.23278519, -0.02469187,
         0.05763403],
       [-0.33267155, -0.25578872,  0.23278519,  0.56720769,  0.0788092 ,
        -0.09478958],
       [ 0.30085264, -0.2890202 , -0.02469187,  0.0788092 ,  0.75344799,
        -0.06998967],
       [ 0.01742278, -0.18212448,  0.05763403, -0.09478958, -0.06998967,
         0.94645473]])

In [7]:
np.allclose(X @ Xd, np.identity(6))

False

In [8]:
Q,R = qr(X)
Q,R = qr(X, mode='economic')

In [9]:
np.allclose(X, Q @ R)

True

In [10]:
Q

array([[-4.75348124e-01,  3.91594809e-01,  2.23403113e-01,
         2.57773283e-01],
       [ 2.90635661e-02, -9.50520322e-02,  3.37649787e-01,
         4.89796681e-01],
       [ 7.13642396e-04,  2.24492955e-01, -5.25945571e-01,
         7.39207108e-01],
       [ 4.53580437e-01,  1.15031901e-01, -5.75618626e-01,
        -1.30012847e-01],
       [-2.69506488e-01,  7.39041478e-01, -1.23554071e-01,
        -3.45494329e-01],
       [-7.03441540e-01, -4.77291196e-01, -4.61370259e-01,
        -1.04667688e-01]])

In [11]:
R

array([[ 3.74544392, -0.69063861,  0.96961492,  0.82682362],
       [ 0.        , -2.08318198,  0.41962445, -1.39136528],
       [ 0.        ,  0.        ,  1.9567931 , -0.6022094 ],
       [ 0.        ,  0.        ,  0.        ,  0.81850821]])

In [12]:
print(np.allclose(Q.T @ Q, np.identity(Q.shape[1])))
Q.T @ Q

True


array([[ 1.00000000e+00, -1.18747961e-16, -1.39416494e-16,
        -5.71998910e-17],
       [-1.18747961e-16,  1.00000000e+00, -1.47861888e-17,
         1.03796678e-17],
       [-1.39416494e-16, -1.47861888e-17,  1.00000000e+00,
         1.57051349e-16],
       [-5.71998910e-17,  1.03796678e-17,  1.57051349e-16,
         1.00000000e+00]])

In [13]:
# form data from noisy linear model
wtrue = randn(d)
y = X.dot(wtrue) + .1*randn(n)

In [14]:
# solve least squares problem to estimate w
w = solve(R, Q.T @ y)

In [15]:
# how good is our estimate?
norm(w - wtrue)

0.0758803460825663

In [16]:
# compute mean square error
def mse(y,z):
    return sum((y-z)**2)/len(y)
    
mse(y,X.dot(w))

0.01219393260293125

In [17]:
# we can use the numpy.lstsq call instead
w_lstsq = np.linalg.lstsq(X, y, rcond=None)[0]
norm(w_lstsq - w)

1.4936523181711914e-15

# Compute QR by hand

In [18]:
n,d = X.shape 
X0 = X.copy()
R = np.zeros((n,d))
Q = np.zeros((n,n))

# first column of Q points in direction of first column of X
r = norm(X[:,0])
Q[:,0] = X[:,0]/r
Q

array([[-0.47534812,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.02906357,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.00071364,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.45358044,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-0.26950649,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-0.70344154,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ]])

In [19]:
# ensure Q*R matches X on first column
R[0,0] = r

In [20]:
# verify Q*R matches X in first column
(Q@R - X)[:,0]

array([0., 0., 0., 0., 0., 0.])

In [21]:
# now delete that part from X; we've covered it already
X[:,0] -= Q[:,0]*R[0,0]

In [22]:
# verify Q*R + X = X0
np.isclose(Q@R + X, X0)

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [23]:
# eliminate component of other columns in direction of first column of Q 
for j in range(1,d):
    R[0,j] = Q[:,0].dot(X[:,j])
    X[:,j] -= Q[:,0]*R[0,j]
R

array([[ 3.74544392, -0.69063861,  0.96961492,  0.82682362],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

In [24]:
# verify Q*R + X = X0
np.isclose(Q@R + X, X0)

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [25]:
# now for all the columns!
X = X0.copy()
Q *= 0
R *= 0

# compute the QR decomposition
for i in range(d):
    r = norm(X[:,i])
    Q[:,i] = X[:,i]/r
    for j in range(i,d):
        R[i,j] = Q[:,i].dot(X[:,j])
        X[:,j] -= Q[:,i]*R[i,j]
    print("iteration",i,": QR + X = X0?", np.isclose(Q@R + X, X0).all())

iteration 0 : QR + X = X0? True
iteration 1 : QR + X = X0? True
iteration 2 : QR + X = X0? True
iteration 3 : QR + X = X0? True


In [26]:
"""Our very own QR function to compute the economy QR"""
def ourQR(X0):
    X = X0.copy()
    n,d = X.shape
    R = np.zeros((n,d))
    Q = np.zeros((n,n))

    # compute the QR decomposition
    for i in range(d):
        r = norm(X[:,i])
        Q[:,i] = X[:,i]/r
        for j in range(i,d):
            R[i,j] = Q[:,i].dot(X[:,j])
            X[:,j] -= Q[:,i]*R[i,j]
    return Q,R

In [31]:
# solve least squares problem to estimate w
Q,R = ourQR(X0)
w_byhand = solve(R[:d,:d], (Q.T @ y)[:d])

In [32]:
norm(w_byhand - w)

1.3822165187958571e-15